<a href="https://colab.research.google.com/github/zzhining/ml_basic/blob/main/customer_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고객 이탈율 예측 모델

**데이터 셋**
- ID: 고객ID
- gender: 성별
- sales: 총매출액
- days: 거래기간
- frequency: 방문빈도
- discount_freq: 할인권 사용 횟수
- discount_price: 할인 금액
- label: 탈회 여부

**목표**
- 주어진 데이터를 사용하여 고객의 탈회 여부를 예측하는 모델을 만든다. 
- 탈회 여부를 예측하는데 가장 큰 영향을 주는 속성이 무엇인지 알아본다.

**모델**
- DecisionTreeClassifier
- RandomForestClassifier
- XGBClassifier

## 데이터 불러오기

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('https://raw.githubusercontent.com/zzhining/ml_basic/main/dataset/customer_sales.csv')
df.head()

## 데이터 탐색

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
sns.countplot(x = 'gender', hue = 'label', data = df)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))
sns.countplot(x = 'discount_freq', hue = 'label', data = df, ax=ax)

In [ ]:
sns.histplot(x = 'frequency', hue = 'label', kde = True, data = df)

In [ ]:
sns.histplot(x = 'sales', hue = 'label', kde = True, data = df)

In [ ]:
sns.histplot(x = 'days', hue = 'label', kde = True, data = df)

## 데이터 전처리

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.drop(['ID'], axis=1, inplace = True)

In [ ]:
df.head()

## 데이터 분할

In [ ]:
# 변수 지정(독립변수/종속변수)
X = df.iloc[:, :-1]
y = df['label']

In [ ]:
from sklearn.model_selection import train_test_split

# 학습용 데이터와 테스트용 데이터의 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

## DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(random_state=42, max_depth = 3)
tree_model.fit(X_train , y_train)

In [ ]:
y_pred = tree_model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

print("테스트셋의 정확도:{:.2f}".format(accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

# 의사결정나무 모델을 dot 파일로 추출
export_graphviz(tree_model, out_file='tree.dot', class_names= ['keep', 'leave'], \
                feature_names = df.columns[:-1], impurity=True, filled=True)

# dot 파일을 .png로 변환
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png'])

# png 출력
Image(filename = 'tree.png')

In [ ]:
feature_name = df.columns[:-1]
feature_importances = tree_model.feature_importances_
sorted(zip(feature_importances, feature_name), reverse=True)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

# 혼동행렬 생성
cm = confusion_matrix(y_test, y_pred)

# 혼동행렬 시각화
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
from sklearn.metrics import precision_score, recall_score

print('정밀도:{:.2f}, 재현율:{:.2f}'.format(precision_score(y_test, y_pred), recall_score(y_test, y_pred)))

## RandomForestClassifier

In [ ]:
# RandomForestClassifier 라이브러리 불러오기
from sklearn.ensemble import RandomForestClassifier

# 기본 매개변수를 사용하는 RandomForestClassifier 생성
rf = RandomForestClassifier(random_state=1)

# 생성한 랜덤포레스트 객체를 학습용 데이터를 사용하여 학습
rf.fit(X_train, y_train)

In [ ]:
# 학습용 데이터 셋의 정확도
print("train 세트 정확도: {:.3f}".format(rf.score(X_train, y_train)))

# 테스트용 데이터 셋의 정확도
print("test 세트 정확도: {:.3f}".format(rf.score(X_test, y_test)))

In [ ]:
# 혼동행렬
y_pred = rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
print('정밀도:{:.2f}, 재현율:{:.2f}'.format(precision_score(y_test, y_pred), recall_score(y_test, y_pred)))

In [ ]:
feature_importances = rf.feature_importances_
sorted(zip(feature_importances, feature_name), reverse=True)

## xgboost

In [ ]:
# XGBoost 라이브러리 불러오기
import xgboost as xgb

# 기본 매개변수를 사용하는 XGBoost 생성
xgb_classifier = xgb.XGBClassifier(random_state=42)

# 생성한 XGBoost 객체를 학습용 데이터를 사용하여 학습
xgb_classifier.fit(X_train, y_train)

In [ ]:
# 학습용 데이터 셋의 정확도
print("train 세트 정확도: {:.3f}".format(xgb_classifier.score(X_train, y_train)))

# 테스트용 데이터 셋의 정확도
print("test 세트 정확도: {:.3f}".format(xgb_classifier.score(X_test, y_test)))

In [ ]:
# 혼동행렬
y_pred = xgb_classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm_display = ConfusionMatrixDisplay(cm).plot()

print('정밀도:{:.2f}, 재현율:{:.2f}'.format(precision_score(y_test, y_pred), recall_score(y_test, y_pred)))

In [ ]:
from xgboost import XGBClassifier, plot_importance

plot_importance(xgb_classifier)

## VotingClassifier

In [ ]:
# VotingClassifier 라이브러리 불러오기
from sklearn.ensemble import VotingClassifier

# 랜덤포레스트와 XGBoost를 앙상블하는 모델 생성
voting_classifier = VotingClassifier(estimators=[('RF',rf), ('xgb',xgb_classifier)], voting='hard')

# 생성한 VotingClassifier 객체를 학습용 데이터를 사용하여 학습
voting_classifier.fit(X_train , y_train)

In [ ]:
# 학습용 데이터 셋의 정확도
print("train 세트 정확도: {:.3f}".format(voting_classifier.score(X_train, y_train)))

# 테스트용 데이터 셋의 정확도
print("test 세트 정확도: {:.3f}".format(voting_classifier.score(X_test, y_test)))